In [1]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
name = 'RED_nPEG_in_agarose_1x_S4'

logplot = '{}_logplot'.format(name)
Mplot = '{}_Mplot'.format(name)
Dplot = '{}_Dplot'.format(name)
Hplot = '{}_Hplot'.format(name)
Hlogplot = '{}_Hlogplot'.format(name)
Cplot = '{}_Cplot'.format(name)
Tplot = '{}_Tplot'.format(name)
T2plot = '{}_T2plot'.format(name)


conversion = dict()
trajectory = dict()

cut = 4
great = 10000
filtered = False
new_method = True
totvids = 6
tofilt = np.array([])

tau_m = 88
frame_m = 90 #atm I can't go lower than the actual value.

for num in range(1, totvids + 1):
    conversion[num] = (0.16, 9.89, 1)
    trajectory[num] = np.genfromtxt('./Traj_{}_{}.csv'.format(name, num), delimiter =",")
    trajectory[num]=np.delete(trajectory[num],0,1)

parts = dict()
tots = dict()
newtots = dict()
newtots[0] = 0
tlen = dict()
tlength = dict()
tlength[0] = 0

for num in range(1, totvids + 1):
    tots[num] = trajectory[num][-1, 0].astype(np.int64)
    parts[num] = tots[num]
    counter = 1
    newtots[num] = newtots[num-1] + tots[num]

    tlen[num] = trajectory[num].shape[0]
    tlength[num] = tlength[num-1] + tlen[num]

placeholder = np.zeros((tlength[totvids], 11))

for num in range(1, totvids + 1):
    placeholder[tlength[num-1]:tlength[num], :] = trajectory[num]
    placeholder[tlength[num-1]:tlength[num], 0] = placeholder[tlength[num-1]:tlength[num], 0] + newtots[num-1]

In [3]:
p_by_vid = dict()
p_by_vid[0] = 0

for num in range(1, totvids + 1):
    p_by_vid[num] = int(p_by_vid[num-1] + max(trajectory[num][:, 0]))
    
p_by_vid[0] = 1

In [4]:
def fillin2(data):
    """
    Fills in blanks of arrays without shifting frames by the starting frame.  Compare to fillin.

    Input: trajectory dataset from MOSAIC tracking software read into a numpy array
    Output: modified numpy array with missing frames filled in.
    """

    shap = int(max(data[:, 1])) + 1
    shape1 = int(min(data[:, 1]))
    newshap = shap - shape1
    filledin = np.zeros((newshap, 5))
    filledin[0, :] = data[0, :]

    count = 0
    new = 0
    other = 0
    tot = 0

    for num in range(1, newshap):
        if data[num-new, 1]-data[num-new-1, 1] == 1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new, 1]-data[num - new - 1, 1] > 1:
            new = new + 1
            iba = int(data[num - new+1, 1]-data[num - new, 1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            # dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)

        else:
            other = other + 1
        tot = count + new + other

    return filledin

In [5]:
rawframes = placeholder[:, 1]
frames = np.linspace(min(rawframes),max(rawframes), max(rawframes)+1).astype(np.int64)
time = frames / conversion[1][1]

x = dict()
y = dict()

xs = dict()
ys = dict()

dataset = dict()
rawdataset = np.zeros(placeholder.shape)
particles = placeholder[:, 0]
total = int(max(particles))
total1 = total + 1
rawdataset = placeholder[:, :]

fixed = np.zeros(placeholder.shape)
fixed[:, 0:2] = rawdataset[:, 0:2]
fixed[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
fixed[:, 4] = conversion[1][2] * rawdataset[:, 4]

for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    dataset[num] = (fixed[min1:max1+1, 0:5])

I = dict()

for num in range(1, total1):
    #Construct x, y, z
    dataset[num] = fillin2(dataset[num])
    x[num] = np.zeros(frames.shape[0])
    x[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 2]
    y[num] = np.zeros(frames.shape[0])
    y[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 3]

    xs[num] = np.zeros(frames.shape[0])
    xs[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 2]
    ys[num] = np.zeros(frames.shape[0])
    ys[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 3]

In [6]:
pretotal = total1

In [7]:
# #Comment out to exclude filter.

# cutoff = cut

# x1 = dict()
# y1 = dict()

# xs1 = dict()
# ys1 = dict()

# fifties = 0
# nones = 0

# for num in range(1, total1):
#     if np.count_nonzero(x[num]) < cutoff:
#         nones = nones + 1
#     else:
#         fifties = fifties + 1
#         x1[num - nones] = x[num]
#         y1[num - nones] = y[num]

#         xs1[num - nones] = xs[num]
#         ys1[num - nones] = ys[num]

# x = x1
# y = y1

# xs = xs1
# ys = ys1

In [8]:
if pretotal == total1:
    for num in range(1, total1):
        xs[num] = ma.masked_equal(xs[num], 0)
        ys[num] = ma.masked_equal(ys[num], 0)

        x[num] = ma.masked_equal(x[num], 0)
        y[num] = ma.masked_equal(y[num], 0)

else:
    for num in range(1, fifties):
        xs[num] = ma.masked_equal(xs[num], 0)
        ys[num] = ma.masked_equal(ys[num], 0)

        x[num] = ma.masked_equal(x[num], 0)
        y[num] = ma.masked_equal(y[num], 0)

    total1 = fifties + 1

In [9]:
total1

2041

In [10]:
(xs[3])

masked_array(data = [22.052 22.31488 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --],
             mask = [False False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True],
       fill_value = 0.0)

In [11]:
tlength = np.zeros(total1)
if pretotal == total1:
    for num in range(1, total1):
        tlength[num - 1] = ma.count(xs[num])
else:
    for num in range(1, total1):
        tlength[num - 1] = ma.count(xs[num])

In [12]:
tlength[0:100]

array([ 90.,   3.,   2.,  90.,   2.,  90.,  90.,  90.,   2.,  90.,   2.,
        90.,   2.,  90.,  69.,  90.,   2.,  90.,   4.,  90.,  90.,   4.,
        90.,  90.,   3.,   3.,  90.,   2.,   2.,   2.,   2.,   2.,   2.,
         2.,  11.,   2.,   2.,  88.,   2.,   2.,   2.,   2.,   2.,   2.,
         2.,  87.,   2.,   2.,   2.,   2.,   2.,   4.,   2.,   8.,   3.,
         5.,   2.,   2.,   2.,   3.,   2.,  11.,   2.,   2.,   2.,   3.,
         3.,   2.,   2.,   2.,   2.,   2.,   5.,   3.,   4.,   2.,   5.,
         2.,   2.,   2.,   2.,   2.,   2.,   3.,   2.,   2.,   2.,   4.,
         2.,   7.,   2.,   2.,   2.,   3.,   6.,   3.,   4.,   9.,   2.,
         3.])

In [13]:
lenplot = '{}_lenplot'.format(name)

In [14]:
def plot_histogram(length, total, filename):
    
    total1 = total
    hist = length

    hist = [x for x in hist if str(x) != 'nan']
    
    plot, bins = np.histogram(hist, bins=total1)
    width = 1 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
    plt.xlabel('Trajectory lengths', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, total1

In [15]:
hist, total= plot_histogram(tlength, total1, lenplot)
plt.show()

In [16]:
"""
This block plots all trajectories without centering.  The user can adjust the range of x and y directly below.
"""

center_zoom = 0
edge = 41
bottom = 0
maxx = edge - center_zoom
minx = bottom + center_zoom
maxy = edge - center_zoom
miny = bottom + center_zoom
vid_to_plot = 2

def plot_traj(xmax, xmin, ymax, ymin, xts, yts):
    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    for num in range(p_by_vid[vid_to_plot - 1], p_by_vid[vid_to_plot] + 1):
        ax.plot(xts[num], yts[num], linewidth=1, label='Particle {}'.format(num))


    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(xmin, xmax + 0.001, 20)
    ymajor_ticks = np.arange(ymin, ymax + 0.001, 20)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel(r'x ($\mu$m)', fontsize=95)
    ax.set_ylabel(r'x ($\mu$m)', fontsize=95)
    #ax.tick_params(direction='out', pad=16)
    #ax.legend(loc=(0.60, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(0)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(0)))


    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([xmin, xmax])
    plt.gca().set_ylim([ymin, ymax])

    # Save your figure
    plt.savefig('{}.png'.format(T2plot), bbox_inches='tight')

plot_traj(maxx, minx, maxy, miny, xs, ys)
plt.show()

In [17]:
#Comment out to exclude filter.

cutoff = cut

x1 = dict()
y1 = dict()

xs1 = dict()
ys1 = dict()

fifties = 0
nones = 0

for num in range(1, total1):
    if np.count_nonzero(x[num]) < cutoff:
        nones = nones + 1
    else:
        fifties = fifties + 1
        x1[num - nones] = x[num]
        y1[num - nones] = y[num]

        xs1[num - nones] = xs[num]
        ys1[num - nones] = ys[num]

for num in range(1, fifties):
    xs1[num] = ma.masked_equal(xs1[num], 0)
    ys1[num] = ma.masked_equal(ys1[num], 0)

    x1[num] = ma.masked_equal(x1[num], 0)
    y1[num] = ma.masked_equal(y1[num], 0)

In [18]:
"""
This block plots user-defined trajectories, defined by their particle numbers.  The user can also adjust the window by
using the unit block scaling all axes.
"""

unit = 61
NPs = np.array([1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]) + 2
#NPs = filte

maxx = 0.101*unit
minx = -0.101*unit
maxy = 0.101*unit
miny = -0.101*unit
ticks = 0.05*unit
dec = 0


#NPs = filte
#NPs = np.array([30])
xc = dict()
yc = dict()
xcmask = dict()

# Creates figure
fig1 = plt.figure(figsize=(24, 18), dpi=80)
ax1 = fig1.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

for num in range(1, NPs.shape[0]+1):
    lowx = ma.min(xs1[NPs[num-1]])
    highx = ma.max(xs1[NPs[num-1]])
    lowy = ma.min(ys1[NPs[num-1]])
    highy = ma.max(ys1[NPs[num-1]])
    
    xc[NPs[num-1]] = np.array([xs1[NPs[num-1]] - ((highx+lowx)/2)])
    yc[NPs[num-1]] = np.array([ys1[NPs[num-1]] - ((highy+lowy)/2)])
    
    xcmask[NPs[num-1]] = xs1[NPs[num-1]].recordmask
    xc[NPs[num-1]] = ma.array(xc[NPs[num-1]], mask = xcmask[NPs[num-1]])
    yc[NPs[num-1]] = ma.array(yc[NPs[num-1]], mask = xcmask[NPs[num-1]])
    
    ax1.plot(xc[NPs[num-1]][0, :], yc[NPs[num-1]][0, :], linewidth=10, label='Particle {}'.format(NPs[num-1]))


# A few adjustments to prettify the graph
for item in ([ax1.xaxis.label, ax1.yaxis.label] +
             ax1.get_xticklabels() + ax1.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(minx, maxx + 0.00001, ticks)
ymajor_ticks = np.arange(miny, maxy + 0.00001, ticks)

ax1.set_xticks(xmajor_ticks)
ax1.set_yticks(ymajor_ticks)
ax1.title.set_fontsize(70)
ax1.set_xlabel(r'x ($\mu$m)', fontsize=95)
ax1.set_ylabel(r'y ($\mu$m)', fontsize=95)
ax1.tick_params(direction='out', pad=16)
plt.xticks(rotation=-30)
# ax1.legend(loc=(0.60, 0.46), prop={'size': 40})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([minx, maxx])
plt.gca().set_ylim([miny, maxy])

# Save your figure
plt.savefig('{}.png'.format(Tplot), bbox_inches='tight')
plt.show()